In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [28]:
util.startLoop()

In [29]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

Error 1100, reqId -1: Conectividad entre IB y TWS ha sido perdida
Error 1102, reqId -1: Conectividad entre IB y TWS ha sido restaurada - datos mantenidos: Todos los centros de datos est\u00e1n conectados: usfarm.nj; hfarm; jfarm; usfuture; cashfarm; eufarmnj; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefnj.


In [30]:
# Definir el contrato para la acción de Pfizer Inc (PFE)
contract = Stock('PFE', 'NYSE', 'USD')

In [31]:
# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=90)

In [32]:
# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='90 D',
    barSizeSetting='1 hour',
    whatToShow='TRADES',
    useRTH=False
)

In [33]:
# Convertir los datos a DataFrame
df = util.df(bars)


In [34]:
# Obtener noticias relevantes sobre Pfizer Inc (PFE)
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=Pfizer%20OR%20NYSE%20PFE&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()


In [35]:
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")


Titulares de noticias relevantes:
 ['Is Pfizer Inc. (PFE) The Best Cheap Dividend Stock To Buy Right Now?', 'Pfizer quietly made some big DEI changes', 'Pfizer CEO says the drugmaker is prepared to shift manufacturing to the US amid tariff threats', 'Is Now The Right Time To Buy Pfizer Stock?', 'Lawmakers demand answers from RFK Jr. over Trump dinners with pharma execs', "Even your weight loss drugs aren't safe from tariffs", 'Big Pharma and drug middlemen donated record money to get Trump on their side', 'Big Pharma wants to bring back a tax break that could save it $15 billion', 'Is Pfizer Inc. (PFE) the Best Weight Loss Drug Stock to Buy According to Analysts?', 'INVESTIGATION ALERT: Berger Montague PC Investigates Pfizer Inc.’s Board Of Directors For Breach of Fiduciary Duties (NYSE: PFE)', 'INVESTIGATION ALERT: Berger Montague PC Investigates Pfizer Inc.’s Board Of Directors For Breach of Fiduciary Duties (NYSE: PFE)', '国産フルスクラッチLLM「PLaMo 2」が公開 ～100Bから8Bの軽量モデルに、個人は無償で商用利用も可能／商用条件に

In [36]:
# Análisis de sentimiento de noticias
positive_keywords = ['rise', 'growth', 'record', 'profit']
negative_keywords = ['fall', 'loss', 'decline', 'drop']

In [37]:
positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))
impact_factor = 1 + (positive_news - negative_news) * 0.001

In [38]:
# Seleccionar datos y normalizar
data = df[['open', 'high', 'low', 'close']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
scaled_data_adjusted = scaled_data * impact_factor


In [39]:
# Identificar soportes y resistencias
def find_support_resistance(data):
    levels = []
    for i in range(2, len(data) - 2):
        if data[i] < data[i - 1] and data[i] < data[i + 1]:
            levels.append((i, data[i]))  # Soporte
        elif data[i] > data[i - 1] and data[i] > data[i + 1]:
            levels.append((i, data[i]))  # Resistencia
    return levels

In [40]:
support_resistance_levels = find_support_resistance(df['close'].values)
print(f"Niveles de soporte y resistencia detectados: {support_resistance_levels}")

Niveles de soporte y resistencia detectados: [(4, np.float64(27.59)), (6, np.float64(27.35)), (7, np.float64(27.48)), (8, np.float64(27.46)), (9, np.float64(27.5)), (11, np.float64(27.36)), (13, np.float64(27.46)), (16, np.float64(26.84)), (17, np.float64(26.89)), (20, np.float64(26.73)), (21, np.float64(26.82)), (27, np.float64(26.23)), (28, np.float64(26.35)), (31, np.float64(25.98)), (34, np.float64(26.19)), (35, np.float64(26.14)), (41, np.float64(26.72)), (43, np.float64(26.48)), (44, np.float64(26.53)), (45, np.float64(26.5)), (46, np.float64(26.53)), (51, np.float64(24.86)), (52, np.float64(24.91)), (54, np.float64(24.73)), (56, np.float64(25.02)), (58, np.float64(24.71)), (60, np.float64(25.04)), (63, np.float64(24.72)), (65, np.float64(25.05)), (67, np.float64(24.84)), (69, np.float64(25.11)), (72, np.float64(24.8)), (73, np.float64(24.92)), (74, np.float64(24.8)), (76, np.float64(24.95)), (77, np.float64(24.8)), (80, np.float64(25.17)), (82, np.float64(24.94)), (86, np.float6

In [41]:
# Crear secuencias de datos para LSTM
sequence_length = 60
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
    y_train.append(scaled_data_adjusted[i])

x_train, y_train = np.array(x_train), np.array(y_train)

In [42]:
# Definir modelo LSTM
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=4))

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.1397
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0217
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0171
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0132
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0147
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0121
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0118
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0105
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0104
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0110
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0103
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0096
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0090
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0079
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0086
Epoc

In [43]:
# Predicciones
recent_data = scaled_data_adjusted[-sequence_length:]
predictions = []
for _ in range(5):
    x_input = np.array([recent_data])
    predicted_price = model.predict(x_input)
    predictions.append(predicted_price[0])
    recent_data = np.append(recent_data[1:], [predicted_price[0]], axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [44]:
# Desnormalizar predicciones
predictions = scaler.inverse_transform(predictions)
prediction_times = [end_time.replace(hour=10, minute=0, second=0) + timedelta(hours=i) for i in range(1, 6)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)
print(predicted_df)

                                 open       high        low      close
2025-03-19 11:00:00.995978  26.305825  26.389408  26.204627  26.294468
2025-03-19 12:00:00.995978  26.308066  26.390477  26.210556  26.295017
2025-03-19 13:00:00.995978  26.298997  26.379866  26.205335  26.283941
2025-03-19 14:00:00.995978  26.283054  26.362728  26.192408  26.266197
2025-03-19 15:00:00.995978  26.262992  26.342042  26.174410  26.244791


In [45]:
ib.disconnect()